In [4]:
# Import dependencies needed for the project
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

### ETL PROJECT: Trump Tweets vs MEX dB
### Jorge Marquez
### Nestor Matus
### Gonzalo Cienfuegos

### EXTRACT - Exchange Rate (MXP/USD) and Tweets CSVs into DF
### Source: http://www.banxico.org.mx/tipcamb/tipCamMIAction.do?idioma=sp
### Source: http://www.trumptwitterarchive.com/

In [16]:
# Read csv from given route
e_r_data_df = pd.read_csv("Resources/exchange_rate.csv")
#read CSV
csv_file = "./Resources/trump_vs_mex.csv"
# Show dataframe data
e_r_data_df.head(2)

,Fecha,Determinación,Publicación DOF,Para solventar obligaciones
0,NaN,NaN,NaN,NaN
1,02/01/2006,10.6253,10.6344,10.7109


In [17]:
#Create dataframe and display
tweets = pd.read_csv(csv_file, encoding = "ISO-8859-1")
tweets.head(2)

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @VP: Earlier this year President @realDonal...,07/12/2019 22:14,11433.0,0.0,True,1.149800e+18
1,Twitter for iPhone,....companies to come to the USA and to get co...,07/12/2019 12:48,9842.0,49450.0,False,1.149660e+18


### TRANSFORM - Reformat date, columns renaming, slice tweets.

In [18]:
# Transform date with format Y/M/D to Y-M-D 
e_r_data_df["Fecha"] = pd.to_datetime(e_r_data_df.Fecha)
# Clean column without NA values
e_r_data_df = e_r_data_df.dropna(how="any")
#e_r_data_df.dtypes
#depending on Python Version the output for column #4 is different for the next line

In [19]:
#Code lines thar run in Python v.3.6.8:
#e_r_data_df = e_r_data_df[["Fecha", "Para solventar\r\nobligaciones"]]
#e_r_data_df = e_r_data_df.rename(columns={'Fecha': 'date', 'Para solventar obligaciones': 'exchange_rate'})
#Code lines that run in Python v3.7.3
e_r_data_df = e_r_data_df[["Fecha", "Para solventar\nobligaciones"]]
e_r_data_df = e_r_data_df.rename(columns={'Fecha': 'date', 'Para solventar\nobligaciones': 'exchange_rate'})
e_r_data_df = e_r_data_df[["date", "exchange_rate"]]
e_r_data_df.head(2)

,date,exchange_rate
1,2006-02-01,10.7109
2,2006-03-01,10.6344


### TRANSFORM (Cont)

In [31]:
#Clean data
tweets_clean = tweets
tweets_clean["created_at"] = pd.to_datetime(tweets_clean["created_at"])
tweets_clean['Date'], tweets_clean['Time'] = tweets_clean['created_at'].dt.normalize(), tweets_clean['created_at'].dt.time
#tweets_clean.set_index('Date').head(2)

### TRANSFORM (Cont)

In [21]:
#Create new dataframe
new_tweets = tweets_clean[["Date", "id_str", "text","retweet_count"]].set_index('id_str')
#rename columns
new_tweets = new_tweets.rename(columns={"text" :"tweet","Date": "date"})
new_tweets.index.names = ['id']
new_tweets = new_tweets.dropna()
#drop retweets
new_tweets =new_tweets[~new_tweets.tweet.str.startswith('RT @')]
#filter by date
new_tweets = new_tweets[(new_tweets['date']>dt.date(2015,1,1)) & (new_tweets['date']<dt.date(2019,8,1))]  
new_tweets.head(2)

C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  if sys.path[0] == '':


,date,tweet,retweet_count
id,,,
1.149660e+18,2019-07-12,....companies to come to the USA and to get co...,9842.0
1.149660e+18,2019-07-12,.....if Mexico produces (which I think they wi...,10001.0


### TRANSFORM (Cont): Dataframes UNION E/R and Trump's Twitter

In [30]:
union_df = pd.merge(new_tweets, e_r_data_df, how='inner', on = 'date').set_index(['date'])
union_df.head(15)

,tweet,retweet_count,exchange_rate
date,,,
2019-07-03,Mexico is doing a far better job than the Demo...,24713.0,19.2597
2019-06-27,Bipartisan Humanitarian Aid Bill for the South...,25170.0,19.2271
2019-06-23,I want to give the Democrats every last chance...,24276.0,18.9804
2019-06-19,Congratulations to President Lopez Obrador  M...,22674.0,19.1924
2019-06-18,Next week ICE will begin the process of removi...,37105.0,19.1487
2019-06-08,I would like to thank the President of Mexico ...,29037.0,19.3116
2019-06-08,Everyone very excited about the new deal with ...,20616.0,19.3116
2019-06-08,MEXICO HAS AGREED TO IMMEDIATELY BEGIN BUYING ...,34068.0,19.3116
2019-06-08,Mexico will try very hard and if they do that ...,17182.0,19.3116


### LOAD: Create database connection and load information into dB

In [23]:
connection_string = "postgres:postgres@localhost:5432/trump_vs_mex_db"
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
# Confirm tables
engine.table_names()

['trump_twitter']

### Load DataFrames into database

In [25]:
union_df.to_sql(name='trump_twitter', con=engine, if_exists='append', index = True)

### VISUALIZATION